In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/metricobjects/master/init.py
import init; init.init(force_download=False); 

In [6]:
import numpy as np
import tensorflow as tf
import sympy as sympy
import itertools
import metricobjects as mo
from progressbar import progressbar as pbar
sympy.init_printing(use_latex=True)
from importlib import reload
reload(mo)
tf.__version__

'2.1.0'

see for reference:

- http://physics.usu.edu/Wheeler/GenRel/Lectures/2Sphere.pdf
- https://digitalcommons.latech.edu/cgi/viewcontent.cgi?article=1008&context=mathematics-senior-capstone-papers

In [7]:
class SySphere(mo.SyModel):
    """
    2D sphered embedded in flat 3D space of radius 1
    """
    def __init__(self):        
        t, p = sympy.symbols('theta phi')

        self.input_vars = [t, p]
        
        self.output_vars = [sympy.sin(t)*sympy.cos(p), 
                            sympy.sin(t)*sympy.sin(p), 
                            sympy.cos(t)
                            ]
        
class TFSphere:
    def __call__(self, u):
        t,p = u[:,0], u[:,1]
        x = tf.sin(t)*tf.cos(p)
        y = tf.sin(t)*tf.sin(p) 
        z = tf.cos(t)
        r = tf.transpose(tf.stack([x,y,z]))
        return r        

In [8]:
tu = tf.Variable(np.array([[+.1,-.9], [-1.,-.9], [-0.1,-1.9], 
                           [0.5,-0.2]]), dtype=np.float32)
nu = tu.numpy()

In [9]:
smo = mo.SympyMetricObjects(SySphere())
tmo = mo.TensorFlowMetricObjects(TFSphere())

check **metric tensor**: $g_{ij}$

In [10]:
sG = smo.get_metric_tensor(nu)
tG = tmo.get_metric_tensor(tu).numpy()
np.allclose(sG, tG)

True

check **inverse metric tensor**: $g^{ij}$


In [11]:
sG_ = smo.get_inverse_metric_tensor(nu)
tG_ = tmo.get_inverse_metric_tensor(tu).numpy()
np.allclose(sG_, tG_)

True

check **metric tensor wrt its inverse**: $g_{ij}g^{jk}=\delta_i^k$

In [12]:
[np.allclose(tG_[0].dot(tG[0]), np.eye(len(smo.model.input_vars)), atol=1e-6) for i in range(len(nu))]

[True, True, True, True]

check **jacobian**


In [13]:
sJ = smo.get_jacobian(nu)
tJ = tmo.get_jacobian(tu)
np.allclose(sJ, tJ)

True

check **derivative of metric tensor wrt input vars**: $\frac{\partial g_{ij}}{\partial u^k}$

In [14]:
sdG = smo.get_dg(nu)
tdG = tmo.get_dg(tu)
np.allclose(sdG, tdG, atol=1e-4)

True

check **christoffel symbols** $\Gamma^i_{jk}$

In [15]:
sC = smo.get_christoffel_symbols(nu)
tC = tmo.get_christoffel_symbols(tu)
np.allclose(sC, tC, atol=1e-4)

True

check **derivative of christoffel symbols wrt input vars**: $\frac{\partial \Gamma^i_{jk}}{\partial u^m}$

In [16]:
sdC = smo.get_dc(nu)
tdC = tmo.get_dc(tu)
np.allclose(sdC, tdC, atol=1e-4)

True

check the **Riemann curvature tensor**: $R^i_{jkm}$

In [17]:
sR  = smo.get_riemann_curvature_tensor(nu)
sRe = smo.get_riemann_curvature_tensor_from_einsteinpy(nu)
tR  = tmo.get_riemann_curvature_tensor(tu)
np.allclose(tR, sR, atol=1e-4), np.allclose(sRe, sR, atol=1e-4)

(True, True)

check the **Ricci tensor**: $R_{jm} = R^i_{jim}$

In [18]:
srt = smo.get_ricci_tensor(nu)
trt = tmo.get_ricci_tensor(tu)
np.allclose(srt, trt, atol=1e-4)

True

check the **Ricci scalar**: $R = g^{jm}R_{jm}$, note that this is an **invariant scalar**

In [19]:
srs = smo.get_ricci_scalar(nu)
trs = tmo.get_ricci_scalar(tu)
np.allclose(srs, trs, atol=1e-4)

True

## Other examples

In [20]:
class SyParaboloid(mo.SyModel):
    def __init__(self):        
        t, p = sympy.symbols('theta phi')
        self.input_vars = [t, p]
        self.output_vars = [t, p, t**2 + p**2]

class TFParaboloid:
    def __call__(self, u):
        t,p = u[:,0], u[:,1]
        x = t
        y = p
        z = t**2+p**2
        r = tf.transpose(tf.stack([x,y,z]))
        return r   

In [29]:
tu = tf.Variable(np.array([[0,0],[.1,.1], [.5,.5], [1,1], [-1,1], [2,2],[3,3]]), dtype=np.float32)
nu = tu.numpy()

In [30]:
smo = mo.SympyMetricObjects(SyParaboloid())
tmo = mo.TensorFlowMetricObjects(TFParaboloid())

In [31]:
tmo.get_ricci_scalar(tu)

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([8.0000000e+00, 6.8587112e+00, 8.8888907e-01, 9.8764926e-02,
       9.8764926e-02, 7.3461719e-03, 1.5005171e-03], dtype=float32)>

In [32]:
smo.get_ricci_scalar(nu)

array([8.00000e+00, 6.85871e+00, 8.88889e-01, 9.87654e-02, 9.87654e-02,
       7.34620e-03, 1.50122e-03])

In [ ]:
class SyEllipsoid(mo.SyModel):
    def __init__(self):        
        t, p = sympy.symbols('theta phi')
        self.input_vars = [t, p]
        self.output_vars = [1*sympy.sin(t)*sympy.cos(p), 
                            2*sympy.sin(t)*sympy.sin(p), 
                            3*sympy.cos(t)
                            ]
        
class TFEllipsoid:
    def __call__(self, u):
        t,p = u[:,0], u[:,1]
        x = 1*tf.sin(t)*tf.cos(p)
        y = 2*tf.sin(t)*tf.sin(p) 
        z = 3*tf.cos(t)
        r = tf.transpose(tf.stack([x,y,z]))
        return r

In [49]:
smo = mo.SympyMetricObjects(SyEllipsoid())
tmo = mo.TensorFlowMetricObjects(TFEllipsoid())

In [50]:
tmo.get_ricci_scalar(tu)

<tf.Tensor: shape=(7,), dtype=float32, numpy=
array([0.        , 3.8646674 , 0.73037195, 0.41818768, 0.41818768,
       0.49998564, 3.3636713 ], dtype=float32)>

In [51]:
smo.get_ricci_scalar(nu)

array([4.5     , 3.86473 , 0.730372, 0.418188, 0.418188, 0.499985,
       3.36368 ])

In [ ]:
class SyQuadratic(mo.SyModel):
    def __init__(self):        
        t, p, r = sympy.symbols('theta phi, ro')
        self.input_vars = [t, p, r]
        self.output_vars = [t**2+p**2, r**2]


class TFQuadratic:
    def __call__(self, u):
        t,p,r = u[:,0], u[:,1], u[:,2]
        x = t**2+p**2
        y = r**2
        r = tf.transpose(tf.stack([x,y]))
        return r 

In [ ]:
smo = mo.SympyMetricObjects(SyQuadratic())
tmo = mo.TensorFlowMetricObjects(TFQuadratic())

In [37]:
tmo.get_ricci_scalar(tu)

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [39]:
tmo.get_christoffel_symbols(tu)

<tf.Tensor: shape=(7, 2, 2, 2), dtype=float32, numpy=
array([[[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]],


       [[[ 9.999999  ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  9.999999  ]]],


       [[[ 2.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  2.        ]]],


       [[[ 1.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  1.        ]]],


       [[[-1.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  1.        ]]],


       [[[ 0.5       ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.5       ]]],


       [[[ 0.33333334,